In [13]:
import numpy as np

def get_adsorption_heights_per_atom(contcar_path, adsorbate_indices, surface_indices):
    """
    Computes adsorption heights for each adsorbate atom, defined as the vertical (z) distance
    to the closest surface atom.

    Parameters:
    - contcar_path (str): Path to the VASP CONTCAR file (in direct coordinates).
    - adsorbate_indices (list[int]): Indices of adsorbate atoms (0-based).
    - surface_indices (list[int]): Indices of surface atoms (0-based).

    Returns:
    - List[float]: List of adsorption heights (in Å) for each adsorbate atom.
    """
    with open(contcar_path, 'r') as f:
        lines = f.readlines()

    scale = float(lines[1].strip())
    lattice_vectors = np.array([list(map(float, lines[i].split())) for i in range(2, 5)])
    lattice_vectors *= scale

    # Find element counts and coordinate start
    idx = 5
    while not all(c.isalpha() or c.isspace() for c in lines[idx]):
        idx += 1
    num_atoms = list(map(int, lines[idx + 1].split()))
    total_atoms = sum(num_atoms)

    coord_start_idx = idx + 2
    if lines[coord_start_idx].strip().lower().startswith("selective"):
        coord_start_idx += 1
    if not lines[coord_start_idx].strip().lower().startswith("direct"):
        raise ValueError("Expected 'Direct' coordinates.")
    coord_start_idx += 1

    # Read and convert coordinates
    direct_coords = np.array([
        list(map(float, lines[i].strip().split()[:3]))
        for i in range(coord_start_idx, coord_start_idx + total_atoms)
    ])
    cart_coords = np.dot(direct_coords, lattice_vectors)

    # Surface and adsorbate z-coordinates
    surface_z = cart_coords[surface_indices][:, 2]

    adsorption_heights = []
    for idx in adsorbate_indices:
        z_ads = cart_coords[idx][2]
        z_closest_surface = surface_z[np.argmin(np.abs(surface_z - z_ads))]
        adsorption_heights.append(z_ads - z_closest_surface)

    return adsorption_heights


path = "/BACKUP/database/surface_adsorbates/IrO2/alcohols_aldehydes_ketones_ethers/Methanol/metal/conf_1/CONTCAR"
adsorbate_indices = [120, 121, 122, 123, 124, 125] 
surface_indices = [29, 89, 9, 49, 99, 59, 24, 4, 94, 54, 39, 109, 19, 69, 119, 79, 34, 14, 114, 74]

heights = get_adsorption_heights_per_atom(path, adsorbate_indices, surface_indices)
for i, h in zip(adsorbate_indices, heights):
    print(f"Atom {i}: adsorption height = {h:.3f} Å")

Atom 120: adsorption height = 1.900 Å
Atom 121: adsorption height = 0.722 Å
Atom 122: adsorption height = 1.551 Å
Atom 123: adsorption height = 0.793 Å
Atom 124: adsorption height = 2.495 Å
Atom 125: adsorption height = 2.490 Å


In [15]:
import numpy as np

def get_adsorption_heights_per_atom(contcar_path, adsorbate_indices, surface_indices):
    """
    Computes adsorption heights for each adsorbate atom, defined as the vertical (z) distance
    to the closest surface atom, and prints the closest surface atom for validation.

    Parameters:
    - contcar_path (str): Path to the VASP CONTCAR file (in direct coordinates).
    - adsorbate_indices (list[int]): Indices of adsorbate atoms (0-based).
    - surface_indices (list[int]): Indices of surface atoms (0-based).

    Returns:
    - List[float]: List of adsorption heights (in Å) for each adsorbate atom.
    """
    with open(contcar_path, 'r') as f:
        lines = f.readlines()

    scale = float(lines[1].strip())
    lattice_vectors = np.array([list(map(float, lines[i].split())) for i in range(2, 5)])
    lattice_vectors *= scale

    # Find where coordinates start
    idx = 5
    while not all(c.isalpha() or c.isspace() for c in lines[idx]):
        idx += 1
    num_atoms = list(map(int, lines[idx + 1].split()))
    total_atoms = sum(num_atoms)

    coord_start_idx = idx + 2
    if lines[coord_start_idx].strip().lower().startswith("selective"):
        coord_start_idx += 1
    if not lines[coord_start_idx].strip().lower().startswith("direct"):
        raise ValueError("Expected 'Direct' coordinates.")
    coord_start_idx += 1

    # Read and convert coordinates
    direct_coords = np.array([
        list(map(float, lines[i].strip().split()[:3]))
        for i in range(coord_start_idx, coord_start_idx + total_atoms)
    ])
    cart_coords = np.dot(direct_coords, lattice_vectors)

    adsorption_heights = []

    print(f"{'Ads Atom':>8} {'Z_ads (Å)':>10} {'Closest Surf Atom':>18} {'Z_surf (Å)':>12} {'Height (Å)':>12}")
    print("-" * 65)

    surface_z = cart_coords[surface_indices][:, 2]

    for ads_idx in adsorbate_indices:
        z_ads = cart_coords[ads_idx][2]
        deltas = np.abs(surface_z - z_ads)
        min_idx_local = np.argmin(deltas)
        z_closest = surface_z[min_idx_local]
        surf_idx = surface_indices[min_idx_local]
        height = z_ads - z_closest
        adsorption_heights.append(height)

        print(f"{ads_idx:8} {z_ads:10.3f} {surf_idx:18} {z_closest:12.3f} {height:12.3f}")

    return adsorption_heights

path = "/BACKUP/database/surface_adsorbates/IrO2/alcohols_aldehydes_ketones_ethers/Methanol/metal/conf_1/CONTCAR"
adsorbate_indices = [120, 121, 122, 123, 124, 125] 
surface_indices = [29, 89, 9, 49, 99, 59, 24, 4, 94, 54, 39, 109, 19, 69, 119, 79, 34, 14, 114, 74]

heights = get_adsorption_heights_per_atom(path, adsorbate_indices, surface_indices)


Ads Atom  Z_ads (Å)  Closest Surf Atom   Z_surf (Å)   Height (Å)
-----------------------------------------------------------------
     120     17.181                109       15.281        1.900
     121     16.003                109       15.281        0.722
     122     16.832                109       15.281        1.551
     123     16.073                109       15.281        0.793
     124     17.776                109       15.281        2.495
     125     17.770                109       15.281        2.490


In [ ]:
import numpy as np

def get_adsorption_heights_plane_fit(contcar_path, adsorbate_indices, surface_indices):
    """
    Computes adsorption heights as perpendicular distances from each adsorbate atom
    to the best-fit plane through the surface metal atoms.

    Parameters:
    - contcar_path (str): Path to VASP CONTCAR (in direct coordinates).
    - adsorbate_indices (list[int]): Indices of adsorbate atoms.
    - surface_indices (list[int]): Indices of surface metal atoms.

    Returns:
    - List[float]: Adsorption heights (in Å) for each adsorbate atom.
    """
    # --- Load CONTCAR and convert to Cartesian
    with open(contcar_path, 'r') as f:
        lines = f.readlines()

    scale = float(lines[1].strip())
    lattice = np.array([list(map(float, lines[i].split())) for i in range(2, 5)]) * scale

    idx = 5
    while not all(c.isalpha() or c.isspace() for c in lines[idx]):
        idx += 1
    num_atoms = list(map(int, lines[idx + 1].split()))
    total_atoms = sum(num_atoms)

    coord_start_idx = idx + 2
    if lines[coord_start_idx].strip().lower().startswith("selective"):
        coord_start_idx += 1
    if not lines[coord_start_idx].strip().lower().startswith("direct"):
        raise ValueError("Expected 'Direct' coordinates.")
    coord_start_idx += 1

    direct_coords = np.array([
        list(map(float, lines[i].split()[:3]))
        for i in range(coord_start_idx, coord_start_idx + total_atoms)
    ])
    cart_coords = direct_coords @ lattice

    # --- Fit plane to surface metal atoms
    surf_coords = cart_coords[surface_indices]
    centroid = np.mean(surf_coords, axis=0)
    shifted = surf_coords - centroid
    _, _, vh = np.linalg.svd(shifted)
    normal = vh[-1]  # Normal to the plane

    # --- Compute perpendicular distance for each adsorbate atom
    heights = []
    print(f"{'Atom':>8} {'Height (Å)':>12}")
    print("-" * 22)
    for i in adsorbate_indices:
        point = cart_coords[i]
        vec = point - centroid
        height = np.dot(vec, normal)  # Signed distance to plane
        heights.append(height)
        print(f"{i:8} {height:12.3f}")

    return heights

path = "/BACKUP/database/surface_adsorbates/IrO2/alcohols_aldehydes_ketones_ethers/Methanol/oxygen/conf_0/CONTCAR"
adsorbate_indices = [120, 121, 122, 123, 124, 125] 
surface_indices = [29, 9, 24, 4, 39, 19, 34, 14]

heights = get_adsorption_heights_per_atom(path, adsorbate_indices, surface_indices)


Ads Atom  Z_ads (Å)  Closest Surf Atom   Z_surf (Å)   Height (Å)
-----------------------------------------------------------------
     120     18.157                 19       14.071        4.086
     121     19.554                 19       14.071        5.483
     122     18.016                 19       14.071        3.945
     123     19.722                 19       14.071        5.651
     124     17.549                 19       14.071        3.478
     125     17.770                 19       14.071        3.699


In [ ]:
import numpy as np

def get_adsorption_heights_plane_fit(contcar_path, adsorbate_indices, surface_indices):
    """
    Computes adsorption heights as perpendicular distances from each adsorbate atom
    to the best-fit plane through the surface metal atoms.

    Parameters:
    - contcar_path (str): Path to VASP CONTCAR (in direct coordinates).
    - adsorbate_indices (list[int]): Indices of adsorbate atoms (0-based).
    - surface_indices (list[int]): Indices of surface metal atoms (0-based).

    Returns:
    - dict[int, float]: Mapping from adsorbate atom index to adsorption height (in Å).
    """
    with open(contcar_path, 'r') as f:
        lines = f.readlines()

    scale = float(lines[1].strip())
    lattice = np.array([list(map(float, lines[i].split())) for i in range(2, 5)]) * scale

    idx = 5
    while not all(c.isalpha() or c.isspace() for c in lines[idx]):
        idx += 1
    num_atoms = list(map(int, lines[idx + 1].split()))
    total_atoms = sum(num_atoms)

    coord_start_idx = idx + 2
    if lines[coord_start_idx].strip().lower().startswith("selective"):
        coord_start_idx += 1
    if not lines[coord_start_idx].strip().lower().startswith("direct"):
        raise ValueError("Expected 'Direct' coordinates.")
    coord_start_idx += 1

    direct_coords = np.array([
        list(map(float, lines[i].split()[:3]))
        for i in range(coord_start_idx, coord_start_idx + total_atoms)
    ])
    cart_coords = direct_coords @ lattice

    # Fit plane to surface metal atoms
    surf_coords = cart_coords[surface_indices]
    centroid = np.mean(surf_coords, axis=0)
    shifted = surf_coords - centroid
    _, _, vh = np.linalg.svd(shifted)
    normal = vh[-1]

    # Compute adsorption heights
    height_dict = {}

    for i in adsorbate_indices:
        point = cart_coords[i]
        vec = point - centroid
        height = np.dot(vec, normal)
        height_dict[i] = height

    return height_dict


path = "/BACKUP/database/surface_adsorbates/IrO2/alcohols_aldehydes_ketones_ethers/Methanol/oxygen/conf_0/CONTCAR"
adsorbate_indices = [120, 121, 122, 123, 124, 125] 
surface_indices = [29, 9, 24, 4, 39, 19, 34, 14]

adsorption_heights = get_adsorption_heights_plane_fit(path, adsorbate_indices, surface_indices)

print(min(adsorption_heights.values()))

3.521258111881781


In [ ]:
def get_surface_indices_from_graph(graph, METALS, cart_coords):
    """
    Identifies the surface metal atoms from the graph representation.

    Parameters:
    - graph: The graph object containing node information.
    - METALS: List of metal elements.
    - cart_coords: Cartesian coordinates of all atoms.

    Returns:
    - List of surface indices (indices of metal atoms on the surface).
    """
    surface_indices = []

    # Loop through all nodes in the graph
    for node_idx, element in enumerate(graph.elem):
        if element in METALS:
            # Get atom index corresponding to this node
            atom_idx = graph.idx[node_idx]
            z_coord = cart_coords[atom_idx][2]

            # Check if this metal atom is part of the surface (topmost atoms)
            # For simplicity, we assume the surface atoms have the largest z-coordinates
            if z_coord == max(cart_coords[graph.idx[i]][2] for i in range(len(cart_coords))):
                surface_indices.append(atom_idx)

    return surface_indices

def get_adsorption_heights_plane_fit(path, graph, atoms, METALS):
    """
    Computes adsorption heights as perpendicular distances from each adsorbate atom
    to the best-fit plane through the surface metal atoms.

    Parameters:
    - path (str): Path to input file (used to find CONTCAR).
    - graph: A graph object that has `adsorbate_indices` attribute (list[int]).
    - atoms: List of all atom symbols (used to distinguish gas-phase systems).
    - METALS: List of metal elements.

    Returns:
    - dict[int, float]: Mapping from adsorbate atom index to adsorption height in Å.
    """

    adsorbate_indices = graph.adsorbate_indices

    # Default to 0 height for non-adsorbed systems
    graph.ads_height = 0

    if (len(adsorbate_indices) == 0) or (len(adsorbate_indices) == len(atoms)):
        return {}

    contcar_path = os.path.join(os.path.dirname(path), "CONTCAR")

    with open(contcar_path, 'r') as f:
        lines = f.readlines()

    # Parse lattice
    scale = float(lines[1].strip())
    lattice = np.array([list(map(float, lines[i].split())) for i in range(2, 5)]) * scale

    # Identify coordinate block
    idx = 5
    while not all(c.isalpha() or c.isspace() for c in lines[idx]):
        idx += 1
    num_atoms = list(map(int, lines[idx + 1].split()))
    total_atoms = sum(num_atoms)

    coord_start_idx = idx + 2
    if lines[coord_start_idx].strip().lower().startswith("selective"):
        coord_start_idx += 1
    if not lines[coord_start_idx].strip().lower().startswith("direct"):
        raise ValueError("Expected 'Direct' coordinates.")
    coord_start_idx += 1

    # Convert to Cartesian coordinates
    direct_coords = np.array([
        list(map(float, lines[i].split()[:3]))
        for i in range(coord_start_idx, coord_start_idx + total_atoms)
    ])
    cart_coords = direct_coords @ lattice

    # Get surface indices from graph
    surface_indices = get_surface_indices_from_graph(graph, METALS, cart_coords)

    # Fit best-fit plane through surface metal atoms
    surf_coords = cart_coords[surface_indices]
    centroid = np.mean(surf_coords, axis=0)
    shifted = surf_coords - centroid
    _, _, vh = np.linalg.svd(shifted)
    normal = vh[-1]

    # Ensure normal points "upward" in +z direction
    if normal[2] < 0:
        normal = -normal

    # Calculate heights for each adsorbate atom
    height_dict = {}
    for i in adsorbate_indices:
        vec = cart_coords[i] - centroid
        height = np.dot(vec, normal)
        height_dict[i] = height

    # Attach minimum adsorption height
    if height_dict:
        graph.ads_height = min(height_dict.values())

    return height_dict

In [ ]:
def get_adsorption_heights_plane_fit(path, graph, surface_indices):
    """
    Computes adsorption heights as perpendicular distances from each adsorbate atom
    to the best-fit plane through the surface metal atoms.

    Parameters:
    - contcar_path (str): Path to VASP CONTCAR (in direct coordinates).
    - adsorbate_indices (list[int]): Indices of adsorbate atoms.
    - surface_indices (list[int]): Indices of surface metal atoms (ONLY METAL ATOMS).

    Returns:
    - dict[int, float]: Mapping from adsorbate atom index to adsorption height (in Å).
    """

    adsorbate_indices = graph.adsorbate_indices

    graph.ads_height = 0

    # For gas phase or slab, no adsorption height calculation needed
    if (graph.type == "gas") or (graph.type == "slab"):
        return {}
    
    contcar_path = os.path.join(os.path.dirname(path), "CONTCAR")

    with open(contcar_path, 'r') as f:
        lines = f.readlines()

    scale = float(lines[1].strip())
    lattice = np.array([list(map(float, lines[i].split())) for i in range(2, 5)]) * scale

    idx = 5
    while not all(c.isalpha() or c.isspace() for c in lines[idx]):
        idx += 1
    num_atoms = list(map(int, lines[idx + 1].split()))
    total_atoms = sum(num_atoms)

    coord_start_idx = idx + 2
    if lines[coord_start_idx].strip().lower().startswith("selective"):
        coord_start_idx += 1
    if not lines[coord_start_idx].strip().lower().startswith("direct"):
        raise ValueError("Expected 'Direct' coordinates.")
    coord_start_idx += 1

    direct_coords = np.array([
        list(map(float, lines[i].split()[:3]))
        for i in range(coord_start_idx, coord_start_idx + total_atoms)
    ])
    cart_coords = direct_coords @ lattice

    # Fit plane to surface metal atoms
    surf_coords = cart_coords[surface_indices]
    centroid = np.mean(surf_coords, axis=0)
    shifted = surf_coords - centroid
    _, _, vh = np.linalg.svd(shifted)
    normal = vh[-1]

    # Compute adsorption heights
    height_dict = {}

    for i in adsorbate_indices:
        point = cart_coords[i]
        vec = point - centroid
        height = np.dot(vec, normal)
        height_dict[i] = height
    
    # Attach minimum adsorption height as label to graph
    graph.ads_height = min(height_dict.values()) if height_dict else 0

    return height_dict

In [ ]:
 def process(self):  
        db = connect(self.ase_database_path)    
        args = []
        for row in db.select(self.db_key):
            args.append(row)

        def process_batch(batch_args):
            with mp.Pool(mp.cpu_count()) as pool:
                return pool.map(self.row_to_data, batch_args)

        batch_size = 2000  # Adjust based on your memory constraints
        data_list = []
        for i in range(0, len(args), batch_size):
            print("Processing batch {} to {} ...".format(i, i + batch_size))
            batch_args = args[i:i + batch_size]
            data_list.extend(deepcopy(process_batch(batch_args)))  # deepcopy to avoid memory issues
        # https://ppwwyyxx.com/blog/2022/Demystify-RAM-Usage-in-Multiprocess-DataLoader/

In [ ]:
import multiprocessing as mp
from functools import partial
from copy import deepcopy

def process(self):
    """
    Process the VASP directory into PyG-compatible graphs using atoms_to_pyg.
    Uses multiprocessing to improve speed on large datasets.
    """
    vasp_files = self.raw_file_names

    if not vasp_files:
        print("No VASP files found to process.")
        return

    # Split paths into batches
    batch_size = 2000  # Adjust based on system memory
    data_list = []

    def process_batch(batch_paths):
        with mp.Pool(self.ncores) as pool:
            process_fn = partial(self.atoms_to_data, graph_params=self.graph_params)
            return pool.map(process_fn, batch_paths)

    for i in range(0, len(vasp_files), batch_size):
        print(f"Processing batch {i} to {i + batch_size} ...")
        batch_paths = vasp_files[i:i + batch_size]
        batch_data = process_batch(batch_paths)

        # Filter out None and avoid memory sharing issues
        batch_data_clean = [deepcopy(d) for d in batch_data if d is not None]
        data_list.extend(batch_data_clean)


